In [1]:
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
import json
import torch
import pickle
import configparser
from ast import literal_eval
from hiko.dask import daskify
from sklearn.metrics import accuracy_score, roc_auc_score
from torch import nn

# Read in config
config = configparser.ConfigParser()
config.read('../ml_models_for_airflow/dbs3_config.ini')

db_engine = create_engine(config['AIRFLOW']['postgres_conn'])

pairs_mapping = literal_eval(config['MODEL']['pairs_mapping'])
pairs = tuple(pairs_mapping.values())

In [34]:
NUM_EPOCHS = 100
GRAD_CLIPPING_VAL = 1
BATCH_SIZE = 64
INIT_LR = 10

In [3]:
from training_data_loader import Dataset
from torch.utils import data

In [4]:
from transformer import Transformer

In [5]:
full_data_set = Dataset(config_location='../ml_models_for_airflow/dbs3_config.ini',
                      pairs=pairs,
                      seq_lenght=16,
                      num_features=3)

In [31]:
train_set_size = int(len(full_data_set)*0.85)
test_set_size = len(full_data_set) - train_set_size

In [32]:
trainset, valset = data.random_split(full_data_set,
                                     [train_set_size, test_set_size]
                                    )

In [35]:
train_generator = data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
test_generator = data.DataLoader(valset, batch_size=len(valset), shuffle=True, num_workers=4)

In [36]:
transformer = Transformer(emb=16,
                          heads=12,
                          depth=2,
                          num_features=3,
                          interpolation_factor=10,
                          dropout=0.2
                          )

In [37]:
optimizer = torch.optim.Adam(transformer.parameters(), lr=1e-5)
learning_rate_scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lambda i: min(i / (INIT_LR / BATCH_SIZE), 1.0))

In [38]:
transformer.apply(Transformer.init_weights)

Transformer(
  (time_series_features_encoding): Conv1d(3, 3, kernel_size=(1,), stride=(1,), bias=False)
  (pos_embedding): Embedding(16, 16)
  (transformer_blocks): Sequential(
    (0): TransformerBlock(
      (attention): SelfAttention(
        (tokeys): Linear(in_features=16, out_features=192, bias=False)
        (toqueries): Linear(in_features=16, out_features=192, bias=False)
        (tovalues): Linear(in_features=16, out_features=192, bias=False)
        (unifyheads): Linear(in_features=192, out_features=16, bias=True)
      )
      (norm1): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
      (norm2): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
      (feed_forward_layers): Sequential(
        (0): Linear(in_features=16, out_features=64, bias=True)
        (1): ReLU()
        (2): Linear(in_features=64, out_features=16, bias=True)
      )
      (dropout): Dropout(p=0.2, inplace=False)
    )
    (1): TransformerBlock(
      (attention): SelfAttention(
        (tokeys

In [39]:
criterion = torch.nn.BCELoss()
track_epoch_loss = []
train_auc = []
test_auc = []

In [ ]:
for ep in range(NUM_EPOCHS):
    transformer.train()
    epoch_loss = 0
    temp_train_auc = 0
    
    for train_x, train_y in train_generator:
        
        predictions = transformer(train_x)
        loss = criterion(predictions, train_y.view(-1, 1))
        epoch_loss += loss.item()
        temp_train_auc += roc_auc_score(
            train_y.numpy(), predictions.detach().numpy())
        
        optimizer.zero_grad()
        loss.backward()
        
        # clip up
        #torch.nn.utils.clip_grad_norm_(transformer.parameters(), GRAD_CLIPPING_VAL)

        optimizer.step()
        #learning_rate_scheduler.step()
    
    train_auc.append(temp_train_auc/len(train_generator))
    print('train auc:', train_auc[-1], ' epoch:', ep)
    
    with torch.no_grad():
        transformer.eval()
        temp_test_auc = 0
        for test_x, test_y in test_generator:
            predictions = transformer(test_x)
            temp_test_auc += roc_auc_score(
                test_y.numpy(), predictions.numpy())

    test_auc.append(temp_test_auc/len(test_generator))
    print('test auc:', test_auc[-1], ' epoch:', ep)